In [7]:
a = seq(1, 7)
b = rep(2, 7)
a
b

[1] 1 2 3 4 5 6 7

[1] 2 2 2 2 2 2 2

# Training a DNN (Keras) using accelertion data

#### Imports:

In [114]:
library(data.table)
library(keras)
install_keras()


Installation complete.



#### Import data:

In [115]:
data <- t(fread(file = '../Data/centred_acc_train_data.csv', header = TRUE))
rownames(data) <- NULL

#### Balance the dataset for model:

In [132]:
set.seed(3)

In [133]:
dive_ix <- which(data[,ncol(data)]==1)  # indicies of dive rows
no_dive_ix <- which(data[,ncol(data)]==0)  # indicies of non-dive rows
no_dive_ix_samp <- sample(no_dive_ix, length(dive_ix))  # sample the same no of non-dive rows that are in dive rows 
balanced.data.ix <- sample(c(dive_ix, no_dive_ix_samp))  # concatenate and shuffle the indicies
balanced.data <- data[balanced.data.ix,]  # subset data
all.zeros <- data[-balanced.data.ix,]

#### Extract explanatory and response variables and subset some indicies for training:

In [134]:
# Extract explanatory and response variables
exp <- balanced.data[,-ncol(balanced.data)]
resp <- balanced.data[,ncol(balanced.data)]

# Random half for training
training <- sample(nrow(exp), nrow(exp)*2/3)  # 2/3 training 1/3 test

# Set data
train.x <- exp[training,]
train.y <- resp[training]
test.x <- exp[-training,]
test.y <- resp[-training]

#### Building model:

In [135]:
#784

In [136]:
# Alternative simple classification MLP with dropout
classify_model <- keras_model_sequential()
# 2048 inputs into 784 hidden units (with dropout to help generalisation)
classify_model %>%
  layer_dense(units = 200, input_shape = ncol(train.x), activation = 'relu') %>% # Size of layer
  layer_dropout(rate=0.2) %>%                 # Apply dropout to nodes in layer
  layer_dense(units = 200, activation = 'relu') %>% # Size of layer
  layer_dropout(rate=0.2) %>%                 # Apply dropout to nodes in layer
  layer_dense(units = 1, activation = 'sigmoid') # final layer is is single softmax output

#compiling the defined model with metric = accuracy and optimiser as adam.
classify_model %>% compile(
  optimizer = 'rmsprop',
  loss = 'binary_crossentropy',
  metrics = c('accuracy')
)

#### Fit Model:

In [137]:
classify_model %>% fit(train.x, train.y, epochs = 200, batch_size = 128)

#### Evaluate model:

In [156]:
cat("\nEvaluation:")
classify_model %>% evaluate(test.x, test.y, batch_size = 128) 

# Confusion matrix (?)
cat("\nConfusion Matrix:")
predictions <- predict(classify_model, test.x) # each row is a picture and each column is a category so each cell is a prob.
table(round(predictions), test.y) # confusion matrix (but why must I round here?)


Evaluation:

loss  accuracy 
0.6147862 0.9415584


Confusion Matrix:

   test.y
     0  1
  0 70  8
  1  1 75

#### Evaluate on all zeros (remainder of dataset)

In [139]:
# Split data
exp2 <- all.zeros[,-ncol(all.zeros)]
resp2 <- all.zeros[,ncol(all.zeros)]

cat("\nEvaluation:")
classify_model %>% evaluate(exp2, resp2, batch_size = 128) 

# Confusion matrix (?)
cat("\nConfusion Matrix:")
predictions2 <- predict(classify_model, exp2) # each row is a picture and each column is a category so each cell is a prob.
table(round(predictions2), resp2) # confusion matrix (but why must I round here?)

# Corr
#cor.test(predictions, resp2)
cat("\nCorrelation:")
sum(round(predictions2)==0)/length(resp2)


Evaluation:

loss  accuracy 
0.2938511 0.9683602


Confusion Matrix:

   resp2
         0
  0 483694
  1  15804


Correlation:

[1] 0.9683602

### All dives

In [151]:
all.ones <- data[data[,ncol(data)]==1,] 

# Split data
exp3 <- all.ones[,-ncol(all.ones)]
resp3 <- all.ones[,ncol(all.ones)]

cat("\nEvaluation:")
classify_model %>% evaluate(exp3, resp3, batch_size = 128) 

# Confusion matrix (?)
cat("\nConfusion Matrix:")
predictions3 <- predict(classify_model, exp3) # each row is a picture and each column is a category so each cell is a prob.
table(round(predictions3), resp3) # confusion matrix (but why must I round here?)

# Corr
#cor.test(predictions, resp2)
cat("\nCorrelation:")
sum(round(predictions3)==1)/length(resp3)


Evaluation:

loss accuracy 
0.400561 0.965368


Confusion Matrix:

   resp3
      1
  0   8
  1 223


Correlation:

[1] 0.965368

In [149]:
dim(all.ones)

[1] 231 154